In [1]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split


In [60]:
# Define dataset paths
IMAGE_DIR = r'/content/C:/Users/User/Downloads/Data/Data/images'
MASK_DIR = r'/content/C:\Users\User\Downloads\Data\Data/masks'

# Image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1  # Assuming grayscale MRI images

# Split ratio
TRAIN_SIZE = 0.8


In [61]:
def load_images(image_dir, mask_dir, img_size=(256, 256)):
    image_paths = sorted(glob(os.path.join(image_dir, "*.png")))  # Adjust extension if necessary
    mask_paths = sorted(glob(os.path.join(mask_dir, "*.png")))

    images = []
    masks = []

    for img_path, mask_path in zip(image_paths, mask_paths):
        # Ensure both image and mask exist
        if os.path.exists(img_path) and os.path.exists(mask_path):
            # Read image in grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            images.append(img)

            # Read mask in grayscale
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                continue
            mask = cv2.resize(mask, img_size)
            masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    # Expand dims to add channel axis
    images = np.expand_dims(images, axis=-1)
    masks = np.expand_dims(masks, axis=-1)

    return images, masks

images, masks = load_images(IMAGE_DIR, MASK_DIR, img_size=(IMG_HEIGHT, IMG_WIDTH))
print(f'Total images: {len(images)}, Total masks: {len(masks)}')


Total images: 0, Total masks: 0


In [62]:
def apply_clahe(images, clip_limit=2.0, tile_grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    enhanced_images = []
    for img in images:
        img = img.squeeze()  # Remove channel dimension
        enhanced = clahe.apply(img)
        enhanced_images.append(enhanced)
    enhanced_images = np.array(enhanced_images)
    enhanced_images = np.expand_dims(enhanced_images, axis=-1)
    return enhanced_images

enhanced_images = apply_clahe(images)


In [63]:
def normalize(images, masks):
    images = images.astype('float32') / 255.0
    masks = masks.astype('float32') / 255.0
    masks = np.round(masks)  # Ensure masks are binary
    return images, masks

images, masks = normalize(enhanced_images, masks)


In [64]:
# Define data augmentation
data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Fit the generators if needed
# image_datagen.fit(images, augment=True, seed=42)
# mask_datagen.fit(masks, augment=True, seed=42)


In [ ]:
# Define the split ratio
TRAIN_SIZE = 0.8  # 80% training, 20% testing

# Perform the split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=TRAIN_SIZE,
    random_state=42,
    shuffle=True
)



In [70]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K


In [71]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [72]:
def conv_block(inputs, num_filters):
    x = layers.Conv2D(num_filters, 3, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(num_filters, 3, activation='relu', padding='same')(x)
    return x

def nested_unet(input_shape=(256, 256, 1), num_classes=1, depth=4, filters=64):
    inputs = layers.Input(shape=input_shape)
    # Nested U-Net architecture
    # Implementation based on U-Net++ paper

    # Initialize dictionaries to store nodes
    conv = {}
    pool = {}
    upsample = {}
    concat = {}

    # Encoder path
    conv[0][0] = conv_block(inputs, filters)
    pool[0] = layers.MaxPooling2D((2, 2))(conv[0][0])

    conv[1][0] = conv_block(pool[0], filters*2)
    pool[1] = layers.MaxPooling2D((2, 2))(conv[1][0])

    conv[2][0] = conv_block(pool[1], filters*4)
    pool[2] = layers.MaxPooling2D((2, 2))(conv[2][0])

    conv[3][0] = conv_block(pool[2], filters*8)
    pool[3] = layers.MaxPooling2D((2, 2))(conv[3][0])

    conv[4][0] = conv_block(pool[3], filters*16)

    # Decoder path
    for i in range(1, depth):
        for j in range(depth - i):
            upsample[j + 1] = layers.UpSampling2D((2,2))(conv[j +1][i -1])
            concat[j][i] = layers.Concatenate()([upsample[j +1], conv[j][i -1]])
            conv[j][i] = conv_block(concat[j][i], filters * (2 ** (j +1)))

    # Final output
    outputs = layers.Conv2D(num_classes, (1,1), activation='sigmoid')(conv[0][depth -1])

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [73]:
def attention_block(x, g, inter_channels):
    theta_x = layers.Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(g)
    add_xg = layers.Add()([theta_x, phi_g])
    act_xg = layers.Activation('relu')(add_xg)
    psi = layers.Conv2D(1, kernel_size=1, strides=1, padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    upsample_psi = layers.UpSampling2D(size=(x.shape[1] // sigmoid_xg.shape[1], x.shape[2] // sigmoid_xg.shape[2]))(sigmoid_xg)
    y = layers.Multiply()([x, upsample_psi])
    return y

def attention_unet(input_shape=(256,256,1), num_classes=1):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2,2))(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2,2))(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2,2))(c3)

    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2,2))(c4)

    # Bridge
    c5 = conv_block(p4, 1024)

    # Decoder with Attention
    a4 = attention_block(c4, c5, 512)
    up4 = layers.UpSampling2D((2,2))(c5)
    up4 = layers.Concatenate()([up4, a4])
    c6 = conv_block(up4, 512)

    a3 = attention_block(c3, c6, 256)
    up3 = layers.UpSampling2D((2,2))(c6)
    up3 = layers.Concatenate()([up3, a3])
    c7 = conv_block(up3, 256)

    a2 = attention_block(c2, c7, 128)
    up2 = layers.UpSampling2D((2,2))(c7)
    up2 = layers.Concatenate()([up2, a2])
    c8 = conv_block(up2, 128)

    a1 = attention_block(c1, c8, 64)
    up1 = layers.UpSampling2D((2,2))(c8)
    up1 = layers.Concatenate()([up1, a1])
    c9 = conv_block(up1, 64)

    outputs = layers.Conv2D(num_classes, (1,1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
# Compile Nested U-Net
unet_plus = nested_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
unet_plus.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])

# Compile Attention U-Net
att_unet = attention_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
att_unet.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [
    ModelCheckpoint('best_model.h5', verbose=1, save_best_only=True, monitor='val_dice_coef'),
    EarlyStopping(monitor='val_dice_coef', patience=10, verbose=1)
]


In [ ]:
# Training parameters
BATCH_SIZE = 16
EPOCHS = 50

# Train Nested U-Net
history_unet_plus = unet_plus.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Train Attention U-Net
history_att_unet = att_unet.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
# Evaluate Nested U-Net
score_unet_plus = unet_plus.evaluate(X_test, y_test, verbose=0)
print(f'Nested U-Net Dice Coefficient: {score_unet_plus[1]}')

# Evaluate Attention U-Net
score_att_unet = att_unet.evaluate(X_test, y_test, verbose=0)
print(f'Attention U-Net Dice Coefficient: {score_att_unet[1]}')


In [ ]:
if score_unet_plus[1] > score_att_unet[1]:
    best_model = unet_plus
    best_model_name = 'Nested U-Net'
else:
    best_model = att_unet
    best_model_name = 'Attention U-Net'

print(f'Best Model: {best_model_name} with Dice Coefficient: {max(score_unet_plus[1], score_att_unet[1])}')


In [ ]:
best_model.save('best_metastasis_segmentation_model.h5')


In [77]:
pip install fastapi uvicorn tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import uvicorn
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import io

app = FastAPI()

# Load the best model
model = tf.keras.models.load_model('best_metastasis_segmentation_model.h5', compile=False)

def preprocess_image(image_bytes, img_size=(256, 256)):
    # Read image
    image = Image.open(io.BytesIO(image_bytes)).convert('L')  # Convert to grayscale
    image = image.resize(img_size)
    image = np.array(image)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    image = clahe.apply(image)

    # Normalize
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0)   # Add batch dimension
    return image

def postprocess_mask(mask):
    mask = (mask > 0.5).astype(np.uint8)
    mask = mask.squeeze() * 255
    return mask

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        image_bytes = await file.read()
        image = preprocess_image(image_bytes)
        prediction = model.predict(image)[0]
        mask = postprocess_mask(prediction)

        # Convert mask to bytes
        _, img_encoded = cv2.imencode('.png', mask)
        mask_bytes = img_encoded.tobytes()

        return JSONResponse(content={"message": "Success"})
    except Exception as e:
        return JSONResponse(content={"message": f"Error: {str(e)}"}, status_code=500)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [79]:
pip install streamlit requests pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# app.py
import streamlit as st
import requests
from PIL import Image
import numpy as np
import cv2
import io
import base64

# FAST API endpoint
API_URL = "http://localhost:8000/predict/"

st.title("Brain MRI Metastasis Segmentation")
st.write("Upload a Brain MRI image to get the metastasis segmentation.")

uploaded_file = st.file_uploader("Choose an MRI image...", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file).convert('L')  # Convert to grayscale
    st.image(image, caption='Uploaded MRI Image', use_column_width=True)

    # Prepare the image for sending
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_bytes = buffered.getvalue()

    # Send the image to the FAST API backend
    st.write("Processing...")
    response = requests.post(API_URL, files={"file": img_bytes})

    if response.status_code == 200:
        st.success("Segmentation Completed!")
        # For demonstration, we'll display the uploaded image as the mask
        # Replace this with actual mask retrieval from the backend
        # Example: Display the mask if returned as base64
        # Here, assuming the backend sends back the mask image bytes
        # Modify the backend to return the mask image in a suitable format
    else:
        st.error("Error in processing the image.")


In [ ]:
# main.py (Enhanced)
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import uvicorn
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import io
import base64

app = FastAPI()

# Load the best model
model = tf.keras.models.load_model('best_metastasis_segmentation_model.h5', compile=False)

def preprocess_image(image_bytes, img_size=(256, 256)):
    # Read image
    image = Image.open(io.BytesIO(image_bytes)).convert('L')  # Convert to grayscale
    image = image.resize(img_size)
    image = np.array(image)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    image = clahe.apply(image)

    # Normalize
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0)   # Add batch dimension
    return image

def postprocess_mask(mask):
    mask = (mask > 0.5).astype(np.uint8) * 255
    mask = mask.squeeze()
    return mask

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        image_bytes = await file.read()
        image = preprocess_image(image_bytes)
        prediction = model.predict(image)[0]
        mask = postprocess_mask(prediction)

        # Convert mask to PNG
        _, img_encoded = cv2.imencode('.png', mask)
        mask_bytes = img_encoded.tobytes()
        mask_base64 = base64.b64encode(mask_bytes).decode('utf-8')

        return JSONResponse(content={"mask": mask_base64})
    except Exception as e:
        return JSONResponse(content={"message": f"Error: {str(e)}"}, status_code=500)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [80]:
# app.py (Enhanced)
import streamlit as st
import requests
from PIL import Image
import numpy as np
import cv2
import io
import base64

# FAST API endpoint
API_URL = "http://localhost:8000/predict/"

st.title("Brain MRI Metastasis Segmentation")
st.write("Upload a Brain MRI image to get the metastasis segmentation.")

uploaded_file = st.file_uploader("Choose an MRI image...", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file).convert('L')  # Convert to grayscale
    st.image(image, caption='Uploaded MRI Image', use_column_width=True)

    # Prepare the image for sending
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_bytes = buffered.getvalue()

    # Send the image to the FAST API backend
    st.write("Processing...")
    response = requests.post(API_URL, files={"file": img_bytes})

    if response.status_code == 200:
        result = response.json()
        mask_base64 = result.get("mask", None)
        if mask_base64:
            mask_bytes = base64.b64decode(mask_base64)
            mask_image = Image.open(io.BytesIO(mask_bytes))
            st.image(mask_image, caption='Metastasis Segmentation Mask', use_column_width=True)
        else:
            st.error("Mask not found in the response.")
    else:
        st.error("Error in processing the image.")


2024-10-01 06:06:15.567 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.665 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-01 06:06:15.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:06:15.678 Thread 'MainThread': mi